# Project Template App - Logistic Regression

In [6]:
!pip install flask-ngrok > /dev/null 2>&1
!pip install pyngrok > /dev/null 2>&1

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
with open('drive/MyDrive/project/static/ngrok_auth.txt') as nf:
    ngrok_auth = nf.read()

In [4]:
!ngrok authtoken $ngrok_auth

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


### Web Server Code

In [13]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import os, joblib
import numpy as np

app = Flask(__name__, static_folder='/content/drive/MyDrive/project/static',
                      template_folder='/content/drive/MyDrive/project/templates')
run_with_ngrok(app)

@app.route('/')
def home():
    menu = {'home':1, 'menu':0}
    return render_template('index.html', menu=menu)

@app.route('/iris', methods=['GET','POST'])
def iris():
    menu = {'home':0, 'menu':0, 'iris':1}
    target_names = ['setosa', 'versicolor', 'virginica']
    if request.method == 'GET':
        return render_template('iris.html', menu=menu, options=target_names)
    else:
        sl = float(request.form['sl'])
        sw = float(request.form['sw'])
        pl = float(request.form['pl'])
        pw = float(request.form['pw'])
        index = int(request.form['species'])
        species = target_names[index]
        test_array = np.array([sl, sw, pl, pw])
        scaler = joblib.load(os.path.join(app.static_folder, 'model/iris_scaler.pkl'))
        test_scaled = scaler.transform(test_array.reshape(1, -1))
        lr = joblib.load(os.path.join(app.static_folder, 'model/iris_best_lr.pkl'))
        pred = lr.predict(test_scaled)
        result = target_names[pred[0]]
        return render_template('iris_res.html', menu=menu, test_data=test_array.tolist(),
                               label=species, result=result)

@app.route('/menu', methods=['GET','POST'])
def menu():
    menu = {'home':0, 'menu':1}
    if request.method == 'GET':
        languages = [
            {'disp':'영어', 'val':'en'},
            {'disp':'일어', 'val':'jp'},
            {'disp':'중국어', 'val':'cn'},
            {'disp':'프랑스어', 'val':'fr'},
            {'disp':'스페인어', 'val':'es'}
        ]
        return render_template('menu.html', menu=menu,
                                options=languages)   # 서버에서 클라이언트로 정보 전달
    else:
        # 사용자가 입력한 정보를 서버가 읽음
        index = request.form['index']
        lang = request.form['lang']
        lyrics = request.form['lyrics']
        #print(lang, '\n', index, '\n', lyrics, sep='')
        # 사용자가 입력한 파일을 읽어서 upload 디렉토리에 저장
        f_image = request.files['image']
        fname = f_image.filename                # 사용자가 입력한 파일 이름
        filename = os.path.join(app.static_folder, 'upload/') + fname
        f_image.save(filename)

        # 모델 실행후 결과를 돌려줌
        result = '독수리 (73.52%)'
        mtime = int(os.stat(filename).st_mtime)
        return render_template('menu_res.html', result=result, menu=menu,
                                fname=fname, mtime=mtime)

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://f8f6-35-233-168-133.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 05:26:46] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 05:26:47] "GET /static/img/logo.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 05:26:47] "GET /static/img/favicon.ico HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 05:26:50] "GET /iris HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 05:27:39] "POST /iris HTTP/1.1" 200 -
